In [1]:
import pandas as pd

In [2]:
data_allforms_training2 = pd.read_csv('all_forms_PROACT_training2.txt', sep='|', header=0, dtype={"feature_value":str, "feature_unit":str}, low_memory = False)
data_allforms_training = pd.read_csv('all_forms_PROACT_training.txt', sep='|', header=0, dtype={"feature_value":str, "feature_unit":str}, low_memory = False)

In [3]:
p2_data_allforms = pd.concat([data_allforms_training, data_allforms_training2])

In [4]:
extent = list(p2_data_allforms["SubjectID"].unique())

In [5]:
print(len(p2_data_allforms["SubjectID"].unique()))

8835


Extract Static Features

In [6]:
p2_demographic = pd.read_csv('demographic.csv')
p2_family_hx = pd.read_csv('family_hx.csv')
p2_als_hx = pd.read_csv('als_hx.csv')
p2_riluzole = pd.read_csv('riluzole.csv')
p2_treatment_group = pd.read_csv('treatment_group.csv')

In [7]:
p2_demographic.query("SubjectID == {0}".format(extent), inplace=True)
p2_family_hx.query("SubjectID == {0}".format(extent), inplace=True)
p2_als_hx.query("SubjectID == {0}".format(extent), inplace=True)
p2_riluzole.query("SubjectID == {0}".format(extent), inplace=True)
p2_treatment_group.query("SubjectID == {0}".format(extent), inplace=True)

In [8]:
p2_static = pd.DataFrame(columns=['SubjectID'])
static_list = [p2_demographic, p2_family_hx, p2_als_hx, p2_riluzole, p2_treatment_group]
for i in static_list :
    df = i
    p2_static = p2_static.merge(df, on='SubjectID', how='outer')

In [9]:
p2_static.to_csv('p2_static_features_with_nans.csv')

Extract Time-resolved Features

In [10]:
p2_alsfrs_3mo = pd.read_csv('alsfrs_total_3mo_meta_slope.csv')
p2_fvc_3mo = pd.read_csv('fvc_3mo_meta.csv')
p2_svc_3mo = pd.read_csv('svc_3mo_meta.csv')
p2_vitals = pd.read_csv('vitals.csv')
p2_weight = pd.read_csv('weight.csv')
p2_bmi = pd.read_csv('bmi.csv')

In [11]:
p2_alsfrs_3mo.query("SubjectID == {0}".format(extent), inplace=True)
p2_fvc_3mo.query("SubjectID == {0}".format(extent), inplace=True)
p2_svc_3mo.query("SubjectID == {0}".format(extent), inplace=True)
p2_vitals.query("SubjectID == {0}".format(extent), inplace=True)
p2_weight.query("SubjectID == {0}".format(extent), inplace=True)
p2_bmi.query("SubjectID == {0}".format(extent), inplace=True)

In [12]:
p2_vitals.query('feature_delta < 92', inplace=True)
p2_weight.query('feature_delta < 92', inplace=True)
p2_bmi.query('feature_delta < 92', inplace=True)

In [13]:
print(len(p2_alsfrs_3mo["SubjectID"].unique()))
print(len(p2_fvc_3mo["SubjectID"].unique()))
print(len(p2_svc_3mo["SubjectID"].unique()))
print(len(p2_vitals["SubjectID"].unique()))
print(len(p2_weight["SubjectID"].unique()))
print(len(p2_bmi["SubjectID"].unique()))

4996
6281
275
5519
6065
3337


In [14]:
p2_alsfrs_3mo.set_index('SubjectID', inplace=True)
p2_fvc_3mo.set_index('SubjectID', inplace=True)
p2_svc_3mo.set_index('SubjectID', inplace=True)
p2_vitals.set_index('SubjectID', inplace=True)
p2_weight.set_index('SubjectID', inplace=True)
p2_bmi.set_index('SubjectID', inplace=True)

In [15]:
p2_alsfrs_3mo = p2_alsfrs_3mo.iloc[:,:4]
p2_fvc_3mo = p2_fvc_3mo.iloc[:,:4]
p2_svc_3mo = p2_svc_3mo.iloc[:,:4]

In [16]:
p2_alsfrs_3mo.to_csv('p_alsfrs_total_3mo_meta_slope.csv')
p2_fvc_3mo.to_csv('p_fvc_3mo_meta.csv')
p2_svc_3mo.to_csv('p_svc_3mo_meta.csv')
p2_vitals.to_csv('p_vitals.csv')
p2_weight.to_csv('p_weight.csv')
p2_bmi.to_csv('p_bmi.csv')

Extract Target variable

In [17]:
p_ALSFRS_slope = pd.read_csv('ALSFRS_slope.csv')

In [18]:
p_ALSFRS_slope.query("SubjectID == {0}".format(extent), inplace=True)

In [19]:
print(len(p_ALSFRS_slope["SubjectID"].unique()))

2281


Preprocessing time-resolved data

1) Weight

In [20]:
p2_weight_s1 = p2_weight.iloc[:,:1].groupby('SubjectID',group_keys=False).agg(['mean', 'max', 'min'])
p2_weight_s1.reset_index()

SubjectID     weight            
                     mean   max   min
0          329  50.380000  52.2  45.8
1          533  51.000000  52.0  50.5
2          649  74.566667  75.6  73.7
3          708  60.450000  63.8  53.9
4          902  62.220000  63.5  61.7
...        ...        ...   ...   ...
6060    999308  72.300000  72.3  72.3
6061    999482  82.750000  85.0  82.0
6062    999823  70.460000  71.9  68.5
6063    999880  68.000000  68.2  67.7
6064    999990  56.720000  56.8  56.7

[6065 rows x 4 columns]

In [21]:
p2_weight_s2 = p2_weight.groupby('SubjectID').agg(['first', 'last'])
p2_weight_s2.reset_index()
p2_weight_s2['weight_change'] = p2_weight_s2.iloc[:,1] - p2_weight_s2.iloc[:,0]
p2_weight_s2['interval'] = p2_weight_s2.iloc[:,3] - p2_weight_s2.iloc[:,2]
p2_weight_s2['weight_slope'] = p2_weight_s2['weight_change']/p2_weight_s2['interval']

In [22]:
p2_weight_stat = p2_weight_s1.merge(p2_weight_s2[['weight_slope']], on = 'SubjectID', how='outer')
p2_weight_stat.columns = p2_weight_stat.columns.droplevel()
p2_weight_stat.columns = ['mean_weight', 'max_weight', 'min_weight', 'slope_weight']

In [23]:
p2_weight_stat.to_csv('p2_weight_stat.csv')

In [24]:
print("mean value of mean_weight is "+str(p2_weight_stat["mean_weight"].mean()))
print("mean value of max_weight is "+str(p2_weight_stat["max_weight"].mean()))
print("mean value of min_weight is "+str(p2_weight_stat["min_weight"].mean()))
print("mean slope of weight is "+str(p2_weight_stat["slope_weight"].mean()))

mean value of mean_weight is 73.16566579870472
mean value of max_weight is 74.1169841379719
mean value of min_weight is 72.07382980260513
mean slope of weight is -0.012173493037189868


2) BMI

In [25]:
p2_bmi_stat = p2_bmi.iloc[:,:1].groupby('SubjectID').mean()
p2_bmi_stat.reset_index()
p2_bmi_stat.columns = ['mean_BMI']

In [26]:
p2_bmi_stat.to_csv('p2_bmi_stat.csv')

In [27]:
print("mean value of mean_bmi is "+str(p2_bmi_stat["mean_BMI"].mean()))

mean value of mean_bmi is 0.0025407999896009895


3) Vitals

In [28]:
p2_vitals_s1 = p2_vitals.iloc[:, [1,2,3,4,5]].groupby('SubjectID').agg(['mean','max', 'min'])
p2_weight_s1.reset_index()
p2_vitals_s1.columns = p2_vitals_s1.columns.droplevel()
p2_vitals_s1.columns = ['mean_bp_d','max_bp_d', 'min_bp_d', 'mean_bp_s', 'max_bp_s', 'min_bp_s', 'mean_pulse', 'max_pulse', 'min_pulse', 'mean_resp_rate', 'max_resp_rate','min_resp_rate', 'mean_temp', 'max_temp', 'min_temp']

In [29]:
p2_vitals_s2 = p2_vitals.groupby('SubjectID').agg(['first', 'last'])

In [30]:
p2_vitals_s2['interval'] = p2_vitals_s2.iloc[:,1] - p2_vitals_s2.iloc[:,0]
p2_vitals_s2['bp_d_slope'] = (p2_vitals_s2.iloc[:,3] - p2_vitals_s2.iloc[:,2])/p2_vitals_s2['interval']
p2_vitals_s2['bp_s_slope'] = (p2_vitals_s2.iloc[:,5] - p2_vitals_s2.iloc[:,4])/p2_vitals_s2['interval']
p2_vitals_s2['pulse_slope'] = (p2_vitals_s2.iloc[:,7] - p2_vitals_s2.iloc[:,6])/p2_vitals_s2['interval']
p2_vitals_s2['resp_rate_slope'] = (p2_vitals_s2.iloc[:,9] - p2_vitals_s2.iloc[:,8])/p2_vitals_s2['interval']
p2_vitals_s2['temp_slope'] = (p2_vitals_s2.iloc[:,11] - p2_vitals_s2.iloc[:,10])/p2_vitals_s2['interval']

In [31]:
p2_vitals_s3 = p2_vitals_s2.iloc[:,13:]
p2_vitals_s3.reset_index()
p2_vitals_s3.columns = p2_vitals_s3.columns.droplevel()
p2_vitals_s3.columns = ['bp_d_slope', 'bp_s_slope', 'pulse_slope', 'resp_rate_slope', 'temp_slope']

In [32]:
p2_vitals_stat = p2_vitals_s1.merge(p2_vitals_s3, on='SubjectID', how='outer')

In [33]:
print("mean slope of bp_diastolic is "+str(p2_vitals_stat["bp_d_slope"].mean()))
print("mean slope of bp_systolic is "+str(p2_vitals_stat["bp_s_slope"].mean()))
print("mean slope of pulse is "+str(p2_vitals_stat["pulse_slope"].mean()))
print("mean slope of repiratory_rate is "+str(p2_vitals_stat["resp_rate_slope"].mean()))
print("mean slope of temperature is "+str(p2_vitals_stat["temp_slope"].mean()))

mean slope of bp_diastolic is -0.005486908160277516
mean slope of bp_systolic is -0.03208867544810356
mean slope of pulse is 0.018748461009642572
mean slope of repiratory_rate is 0.007905405721869778
mean slope of temperature is 0.0006912282390870383


In [34]:
p2_vitals_stat.to_csv('p2_vitals_stat.csv')

4) ALSFRS_3mo

In [35]:
print("mean slope of total alsfrs is "+str(p2_alsfrs_3mo["alsfrs_total_slope"].mean()))
print("mean value of mean_alsfrs_total is "+str(p2_alsfrs_3mo["mean_alsfrs_total"].mean()))
print("mean value of max_alsfrs_total is "+str(p2_alsfrs_3mo["max_alsfrs_total"].mean()))
print("mean value of min_alsfrs_total is "+str(p2_alsfrs_3mo["min_alsfrs_total"].mean()))

mean slope of total alsfrs is -0.028024605618735862
mean value of mean_alsfrs_total is 29.0226814785121
mean value of max_alsfrs_total is 30.388877768875105
mean value of min_alsfrs_total is 27.52341873498799


5) FVC, SVC 3mo

In [36]:
print("mean slope of fvc is "+str(p2_fvc_3mo["fvc_slope"].mean()))
print("mean value of mean_fvc is "+str(p2_fvc_3mo["fvc_mean"].mean()))
print("mean value of max_fvc is "+str(p2_fvc_3mo["fvc_max"].mean()))
print("mean value of min_fvc is "+str(p2_fvc_3mo["fvc_min"].mean()))

mean slope of fvc is -0.09043775205298756
mean value of mean_fvc is 79.2786074404287
mean value of max_fvc is 83.39961789523961
mean value of min_fvc is 74.92182773443719


In [37]:
print("mean slope of svc is "+str(p2_svc_3mo["svc_slope"].mean()))
print("mean value of mean_svc is "+str(p2_svc_3mo["svc_mean"].mean()))
print("mean value of max_svc is "+str(p2_svc_3mo["svc_max"].mean()))
print("mean value of min_svc is "+str(p2_svc_3mo["svc_min"].mean()))

mean slope of svc is -0.09845913731248648
mean value of mean_svc is 84.73484848484846
mean value of max_svc is 90.57454545454546
mean value of min_svc is 78.57818181818182


combine time-resolved features into one dataframe

In [38]:
p2_time_res = pd.DataFrame(columns=['SubjectID'])
time_res_list = [p2_alsfrs_3mo, p2_fvc_3mo, p2_svc_3mo, p2_vitals_stat, p2_weight_stat, p2_bmi_stat]
for i in time_res_list :
    df = i
    p2_time_res = p2_time_res.merge(df, on='SubjectID', how='outer')

In [39]:
p2_time_res.to_csv('p2_time_resolved_features_with_nans.csv')

Merge all features into one dataframe

In [40]:
p2_features_combined = p2_static.merge(p2_time_res, on='SubjectID', how='outer')

In [41]:
p2_features_combined.to_csv('p2_features_combined_with_nans.csv')

Subject count

In [43]:
print(len(p2_static["SubjectID"].unique()))
print(len(p2_time_res["SubjectID"].unique()))
print(len(p2_features_combined["SubjectID"].unique()))

8835
7604
8835
